<a href="https://colab.research.google.com/github/koliby777/pokus-cislo/blob/master/Jazykov%C3%BD_model_CLAUDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Jazykový model CLAUDE**

https://monica.im/share/chat?shareId=QSSCFWegWektmRoB

In [1]:
!wget https://raw.githubusercontent.com/koliby777/pokus-cislo/master/Texty_EU/cestina.txt
with open('cestina.txt', 'r', encoding='utf-8') as f:
    cz_text = f.read()

!wget https://raw.githubusercontent.com/koliby777/pokus-cislo/master/Texty_EU/anglictina.txt
with open('anglictina.txt', 'r', encoding='utf-8') as f:
    en_text = f.read()

# Kontrola délky stažených textů
print(len(cz_text))
print(len(en_text))




--2024-04-27 01:15:24--  https://raw.githubusercontent.com/koliby777/pokus-cislo/master/Texty_EU/cestina.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97673920 (93M) [text/plain]
Saving to: ‘cestina.txt’

cestina.txt         100%[===================>]  93.15M   229MB/s    in 0.4s    

2024-04-27 01:15:33 (229 MB/s) - ‘cestina.txt’ saved [97673920/97673920]

--2024-04-27 01:15:34--  https://raw.githubusercontent.com/koliby777/pokus-cislo/master/Texty_EU/anglictina.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94786880 (

In [2]:
!pip install nltk


In [3]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import nltk
from collections import Counter

# Tokenizace textů
cz_tokens = nltk.word_tokenize(cz_text)
en_tokens = nltk.word_tokenize(en_text)

# Vytvoření slovníku
counter = Counter(cz_tokens + en_tokens)
vocab = sorted(counter, key=counter.get, reverse=True)
vocab_to_idx = {token: idx for idx, token in enumerate(vocab)}

# Převod tokenů na indexy
cz_indices = [vocab_to_idx[token] for token in cz_tokens]
en_indices = [vocab_to_idx[token] for token in en_tokens]

# Vytvoření datových sad
train_data = cz_indices[:int(0.8 * len(cz_indices))] + en_indices[:int(0.8 * len(en_indices))]
val_data = cz_indices[int(0.8 * len(cz_indices)):] + en_indices[int(0.8 * len(en_indices)):]

# Vytvoření datových loaderů
import torch
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(torch.tensor(train_data))
val_dataset = TensorDataset(torch.tensor(val_data))

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


In [5]:
# Kontrola tokenizace
print("První tokeny z českého textu:", cz_tokens[:10])
print("První tokeny z anglického textu:", en_tokens[:10])

# Kontrola slovníku
print("První tokeny ve slovníku:")
for token, idx in list(vocab_to_idx.items())[:10]:
    print(f"{token}: {idx}")

# Kontrola převodu tokenů na indexy
print("První indexy z českých indexovaných sekvencí:", cz_indices[:10])
print("První indexy z anglických indexovaných sekvencí:", en_indices[:10])

# Kontrola datových sad
print("Délka trénovací sady:", len(train_data))
print("Délka validační sady:", len(val_data))

# Kontrola datových loaderů
print("Tvar dávky z trénovacího loaderu:")
for batch in train_loader:
    print(batch[0].shape)
    break

print("Tvar dávky z validačního loaderu:")
for batch in val_loader:
    print(batch[0].shape)
    break


První tokeny z českého textu: ['europarl-v7.cs-en.cs', '0000644', '0002002', '0000144', '00566323652', '11662467504', '014035', '0', 'ustar', 'pkoehn']
První tokeny z anglického textu: ['ho', 'významu', ',', 'vzhledem', 'k', 'tomu', ',', 'že', '70', '%']
První tokeny ve slovníku:
,: 0
.: 1
the: 2
a: 3
to: 4
of: 5
and: 6
in: 7
v: 8
that: 9
První indexy z českých indexovaných sekvencí: [161876, 125213, 125214, 125215, 161877, 125216, 161878, 18854, 125217, 125218]
První indexy z anglických indexovaných sekvencí: [1520, 3441, 0, 1058, 37, 175, 0, 13, 3438, 122]
Délka trénovací sady: 25830795
Délka validační sady: 6457700
Tvar dávky z trénovacího loaderu:
torch.Size([64])
Tvar dávky z validačního loaderu:
torch.Size([64])
